In [27]:
import hashlib
import json
from time import time
from collections import OrderedDict

In [28]:
class Printed:
    """A base class"""
    def __repr__(self):
        return str(self.__dict__)

In [29]:
class Block(Printed):
    """When a block is added to the chain..."""
    def __init__(self, index, previousHash, transactions, proof, time=time()):
        self.index = index
        self.previousHash = previousHash
        self.timestamp = time
        self.transactions = transactions
        self.proof = proof

In [30]:
class Transaction(Printed):
    """When a transaction is added to a block"""
    def __init__(self, sender, receiver, amount):
        self.sender = sender
        self.receiver = receiver
        self.amount = amount

    def to_ordered_dict(self):
        """Converts into dict"""
        return OrderedDict([('sender', self.sender), ('receiver', self.receiver), ('amount', self.amount)])


In [31]:
def hashString(string):
    return hashlib.sha256(string).hexdigest()

def hashBlock(block):
    hashingBlock = block.__dict__.copy()
    hashingBlock['transactions'] = [tx.to_ordered_dict() for tx in hashingBlock['transactions']]
    return hashString(json.dumps(hashingBlock, sort_keys=True).encode())

In [32]:
tx = Transaction('xyz','tou',4)

In [33]:
tx

{'sender': 'xyz', 'receiver': 'tou', 'amount': 4}

In [34]:
fblock = Block(0,'00',[tx],0)

In [35]:
fblock

{'index': 0, 'previousHash': '00', 'timestamp': 1613241245.108352, 'transactions': [{'sender': 'xyz', 'receiver': 'tou', 'amount': 4}], 'proof': 0}

In [36]:
hashBlock(fblock)

'5856b28c8aa417d69cc509cc4d4b25bf7c903d09ce570f6bee015747d7ccc457'

In [37]:
"""Verification: helper methods.."""

class Verification:
    @staticmethod
    def validProof(transactions, lastHash, proof):
        guess = (str([tx.to_ordered_dict() for tx in transactions]) + str(lastHash) + str(proof)).encode()
        _hash = hashString(guess)
        return _hash[0:2] == '00'
        
    @classmethod
    def verifyChain(cls, chain):
        for (index, block) in enumerate(blockchain):
            if index == 0:
                continue
            if block.previousHash != hashBlock(blockchain[index - 1]):
                return False
            if not cls.validProof(block.transactions[:-1], block.previousHash, block.proof):
                print('Invalid proof of work')
                return False
        return True
        


In [94]:


class Blockchain:
    def __init__(self):
        genesis_block = Block(0, '', [], 0, 0)
        self.chain = [genesis_block]
        self.unconfirmedTransaction = []
        self.REWAED = 10
        
    @property
    def chain(self):
        return self.__chain[:]

    # The setter
    @chain.setter
    def chain(self, data):
        self.__chain = data
    
    def proofOfWork(self):
        last_block = self.__chain[-1]
        lastHash = hashBlock(last_block)
        proof = 0
        while not Verification.validProof(self.unconfirmedTransaction, lastHash, proof):
            proof += 1
        return proof

    
    @property    
    def unconfirmed(self):
        return self.unconfirmedTransaction[:]


    def lastChain(self):
        if len(self.__chain) < 1:
            return None
        return self.__chain[-1]

    
    def addTransaction(self, receiver, sender, amount=0.90):
        transaction = Transaction(sender, receiver, amount)
        self.unconfirmedTransaction.append(transaction)
        return True


        
    def addBlcok(self):
        last_block = self.__chain[-1]
        hashed_block = hashBlock(last_block)
        proof = self.proofOfWork()

        reward_transaction = Transaction(
            'MINING', 'receiverAddress', self.REWAED)

        copied_transactions = self.unconfirmedTransaction[:]

        copied_transactions.append(reward_transaction)

        block = Block(len(self.__chain), hashed_block, copied_transactions, proof)

        block.hash = hashBlock(block)

        self.__chain.append(block)
        self.unconfirmedTransaction = []

        return block
    





In [86]:
blockchain = Blockchain()

In [91]:
blockchain.addTransaction("Arry", "Salwa", 19)

True

In [92]:
blockchain.unconfirmed

[{'sender': 'Salwa', 'receiver': 'Anwar', 'amount': 4},
 {'sender': 'Salwa', 'receiver': 'Adam', 'amount': 9},
 {'sender': 'Salwa', 'receiver': 'Arry', 'amount': 19}]

In [93]:
blockchain.addBlcok()

{'index': 1, 'previousHash': '817c94b3a53cf23a06afceb0cbfcd9c71eac120fbf89a47aba1ae8d170d08999', 'timestamp': 1613241245.108352, 'transactions': [{'sender': 'Salwa', 'receiver': 'Anwar', 'amount': 4}, {'sender': 'Salwa', 'receiver': 'Adam', 'amount': 9}, {'sender': 'Salwa', 'receiver': 'Arry', 'amount': 19}, {'sender': 'MINING', 'receiver': 'receiverAddress', 'amount': 10}], 'proof': 185, 'hash': 'e6a4e8e954e22f526c28a695f3b40453ac1861a517ae358b4331e5788472294c'}